In [5]:
import re
import unicodedata
import pandas as pd

In [10]:
class Preprocess:
        
        # --------------------------------------- Constructor --------------------------------------- 
        
        def __init__(self,stopword_list):
            self.data_path = ''
            self.stopword_list = stopword_list
                

        # --------------------------------------- Expand Concatenations --------------------------------------- 
        
        def expand_concatenations(self, word):            
            if not re.match('[a-zA-Z]+', word) or re.match('\d+',word):
                for i in range(len(word)):
                    if not('DEVANAGARI ' in unicodedata.name(word[i])):
                        word = word[:i] if( len(word[i:]) < 2 and not word[i:].isnumeric()) else word[:i] + " " + word[i:]
                        break
            else:
                for i in range(len(word)):
                    if ('DEVANAGARI ' in unicodedata.name(word[i])):
                        word = word[i:] if( len(word[:i]) < 2 and not word[:i].isnumeric() ) else word[:i] + " " + word[i:]
                        break
            return(word)
    
    
        # --------------------------------------- Preprocess --------------------------------------- 

        def clean_text(self,text: str) -> str:
            try:
                if not(isinstance(text, str)): text = str(text)

                #Removing unprintable characters
                text = ''.join(x for x in text if x.isprintable())

                # Cleaning the urls
                text = re.sub(r'https?://\S+|www\.\S+', '', text)

                # Cleaning the html elements
                text = re.sub(r'<.*?>', '', text)

                # Removing the punctuations
                text = re.sub('[!#?,.:";-@#$%^&*_~<>()-]', '', text)


                # Removing stop words
                text = ' '.join([word for word in text.split() if word not in self.stopword_list])

                # Expanding noisy concatenations (Eg: algorithmआणि  -> algorithm आणि ) 
                text = ' '.join([self.expand_concatenations(word) for word in text.split()])

                preprocessed_text = ""

                for word in text.split(): 
                    if (re.match('\d+', word)):
                        if(word.isnumeric()):
                            preprocessed_text = preprocessed_text + '#N' + " "
                    else:
                        if(re.match('[a-zA-Z]+', word)):
                            if not len(word) < 2:
                                preprocessed_text = preprocessed_text + word.lower() + " "
                        else:
                            preprocessed_text = preprocessed_text + word + " "

                return preprocessed_text.strip()
            
            except ValueError as ve:
                print('Error processing:\t',text)
                return ''

In [11]:
pp = Preprocess([])

In [12]:
#sampletext1 = df['text'].sample().values
sampletext2 = 'त्यांना जनतेला पटवून द्यावे लागेल99 '

test_list = ['त्यांना CO2 सीओ२ co२ लागेल99 , Animalत्यांना त्यांनाAnimal Analogy_त्यांना Science२१ १२Number',
             '!@)$&%!#)&$!&$!$His am Atharva ११.२२ Kulkarni 11.22 a B 1 सी']
             
for text in test_list:
    result = pp.clean_text(text)
    print("\nFinal result: \n", result)


Final result: 
 त्यांना co2 सीओ२ co #N लागेल #N animal त्यांना त्यांना animal analogy त्यांना science #N #N number

Final result: 
 his am atharva #N kulkarni #N #N सी


In [ ]:
data = pd.read_csv("../dataset/original-dataset/marathi-training-data.csv")
data

In [ ]:
data['text'] = data.text.apply(lambda x: pp.clean_text(x))
data

In [ ]:
corpus = data.text.values.tolist()